In [55]:
import matplotlib.pyplot as plt
import time

# Tensorflow / Keras
from keras import Model
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Lambda
from keras.layers import Activation
from keras.layers import TimeDistributed
from keras.layers import Input
from keras.optimizers import Adam

from keras.optimizers import SGD

# Data manipulation
import pandas as pd
import numpy as np

# file processing
import os

## Preparing Data

In [31]:
PATH = "test_input"  
file_dir = os.listdir(PATH) 
X_train = list()
y_train = list()
max_len = -1
for file in file_dir:
    loaded_file = np.load(PATH + "/" + file)
    # if the file name ends with "X.npy"
    if file.endswith("ans.npy"):
        y_train.append(loaded_file)
    else:
        X_train.append(loaded_file)
    max_len = max_len if max_len>=loaded_file.shape[0] else loaded_file.shape[0]

# print all the contents of x_train
print(X_train)
# print all the contents of training samples
print("Num. of music: " + str(len(X_train)))
print(X_train[0].shape)
print(y_train[0].shape)

[array([[0.00e+00, 0.00e+00, 0.00e+00, ..., 4.00e+00, 0.00e+00, 0.00e+00],
       [0.00e+00, 0.00e+00, 0.00e+00, ..., 4.00e+00, 1.00e+00, 0.00e+00],
       [0.00e+00, 0.00e+00, 0.00e+00, ..., 4.00e+00, 2.00e+00, 0.00e+00],
       ...,
       [3.63e+03, 0.00e+00, 0.00e+00, ..., 4.00e+00, 1.00e+00, 1.51e+02],
       [3.63e+03, 0.00e+00, 0.00e+00, ..., 4.00e+00, 2.00e+00, 1.51e+02],
       [3.63e+03, 0.00e+00, 0.00e+00, ..., 4.00e+00, 3.00e+00, 1.51e+02]])]
Num. of music: 1
(2420, 42)
(2420, 52)


## Parameters

In [58]:
training_size = len(X_train)
# test_size = len(X_test)
n_song = len(X_train[0])
n_input = X_train[0].shape[0]
n_step = 16

n_features = 42 # Hidden layer num of features
n_note = 52 # Total classes of notes

# Training
epoch = 300
learning_rate = 0.001
training_iters = training_size * epoch  # Loop epoch times on the dataset
batch_size = 64

In [26]:
# # batch, seq_len, features = 16, max_len, 42 (12*3 + 6)
# class MyModel(Model):
#   def __init__(self):
#     super(MyModel, self).__init__()
    
#     self.lstm1 = LSTM(units=52, return_sequences=True, input_shape=(16, 42), stateful=True)
#     self.lstm2 = LSTM(units=52, return_sequences=True, input_shape=(16, 42), stateful=True)
#     # self.d = keras.layers.LSTMCell 

#   def call(self, x):
#     x = self.lstm1(x)
#     x = self.lstm2(x)
#     for _ in range(16)
#     # x= 
#     return x

# # Create an instance of the model
# model = MyModel()

## Additional Functions

In [62]:
def batch_generator():
    None

## Build Neural Network

In [59]:
print("Creating model")
input = Input(shape=(n_step, n_features,), name="Input_step_of_chromagram_data")  

LSTM_forward = LSTM(units=n_note,
                input_shape=(n_step, n_features, ),
                return_sequences=True,
                name="LSTM_forwards")(input)

time_dist = TimeDistributed(Dense(n_note, activation="tanh"), name="Dense_layer_for_each_memory_cell")(LSTM_forward)

softmax = Activation('softmax', name="Softmax")(time_dist)

model = Model(inputs=input, outputs=softmax)

model.compile(loss='categorical_crossentropy',
                optimizer=Adam(learning_rate=learning_rate),
                metrics=['categorical_accuracy'])
print(model.summary())

Creating model
Model: "model_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_step_of_chromagram_da  [(None, 16, 42)]         0         
 ta (InputLayer)                                                 
                                                                 
 LSTM_forwards (LSTM)        (None, 16, 52)            19760     
                                                                 
 Dense_layer_for_each_memory  (None, 16, 52)           2756      
 _cell (TimeDistributed)                                         
                                                                 
 Softmax (Activation)        (None, 16, 52)            0         
                                                                 
Total params: 22,516
Trainable params: 22,516
Non-trainable params: 0
_________________________________________________________________
None


## Train Model

In [ ]:
# train model
model.fit_generator(batch_generator(),
                    steps_per_epoch=(n_input/batch_size),
                    epochs=epoch,
                    verbose=1)
model.save("lstm")

In [ ]:
pred_test = model.predict(X_train)